In [3]:
pip install --user opencv-contrib-python


In [4]:
pip install opencv-python opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [25]:
import cv2
import numpy as np
import csv

# Function to convert a rotation matrix to Euler angles
def rotationMatrixToEulerAngles(R):
    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.degrees(x), np.degrees(y), np.degrees(z)

In [26]:
# Load the baseline CSV data
def load_baseline_csv(csv_file):
    baseline_data = []
    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row
        for row in csv_reader:
            frame_id = int(row[0])
            qr_id = int(row[1])
            qr_2d = eval(row[2])  # Convert string to list
            qr_3d = eval(row[3])  # Convert string to list
            baseline_data.append((frame_id, qr_id, qr_2d, qr_3d))
    return baseline_data

In [27]:
def calculate_center_2d(corners):
    #print("calculate_center_2d " , corners)
    mean_x = 0
    mean_y = 0
    # Each corner has 4 points, so we need to iterate over them
    for point in corners:  # corners[0] contains the points
        mean_x += point[0]
        mean_y += point[1]
    mean_x /= 4
    mean_y /= 4
    return mean_x, mean_y

In [28]:
# Calculate movement commands based on current and target poses
def calculate_movement_commands(current_pose, target_pose, current_distance, target_distance,live_corners,target_2d_corners):
    # Calculate differences in distance, yaw, pitch, roll
    distance_diff = target_distance - current_distance
    yaw_diff = (target_pose[1] - current_pose[0]) % 180
    pitch_diff = target_pose[2] - current_pose[1]
    roll_diff = target_pose[3] - current_pose[2]
    
    live_center_x,live_center_y = calculate_center_2d(live_corners[0][0].tolist())
    target_center_x,target_center_y = calculate_center_2d(target_2d_corners)
    x_diff = live_center_x-target_center_x
    y_diff = live_center_y - target_center_y
    
    # Initialize commands list
    commands = []
     
    # Movement based on distance difference
    if abs(distance_diff) > 1.5:
        if distance_diff > 0:
            commands.append("backward")
        else:
            commands.append("forward")

    if abs(x_diff) > 15:
        if x_diff > 0:
            commands.append("right")
        else:
             commands.append("left")
                
    if abs(y_diff) > 15:
        if y_diff > 0:
            commands.append("down")
        else:
            commands.append("up")
    # Movement based on yaw difference
    if abs(yaw_diff) > 18:
        if yaw_diff > 0:
            commands.append("turn-right")
        else:
            commands.append("turn-left")
    commands.append("In Position")
    return commands if commands else "In Position"

In [29]:
# Main function for processing live video and generating commands
def process_live_video(baseline_data, target_frame_id):
    # Initialize video capture
    cap = cv2.VideoCapture(1)  # Use 0 for default webcam , 1 for external webcam
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    print("Webcam successfully connected.")
    
    # Find the target baseline data
    target_baseline = None
    for baseline_entry in baseline_data:
        if baseline_entry[0] == target_frame_id:
            target_baseline = baseline_entry
            break
    
    if not target_baseline:
        print(f"Error: No baseline data found for frame ID {target_frame_id}.")
        return
    
    target_qr_id = target_baseline[1]
    target_2d_corners = target_baseline[2] # 2d corners ;[(x,y),(x,y),(x,y),(x,y)]
    target_pose = target_baseline[3]  # [distance, yaw, pitch, roll]
    target_distance = target_pose[0]  # Assuming the first element in target_pose is distance

    in_position = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect ArUco markers in the frame
        live_corners, ids, rejected = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    
        if ids is not None:
            for i in range(len(ids)):
                id = ids[i][0]
                if id == target_qr_id:
                    # Estimate pose of the detected marker
                    rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(live_corners[i], marker_length, camera_matrix, dist_coeffs)
                    rvec = rvecs[0]
                    tvec = tvecs[0]  # Translation vector
                    current_distance = np.linalg.norm(tvec)
                    R, _ = cv2.Rodrigues(rvec)
                    current_pose = rotationMatrixToEulerAngles(R)  # [yaw, pitch, roll]
                    
                    # Calculate distance difference
                    distance_diff = target_distance - current_distance
                    
                    # Calculate movement commands
                    command = calculate_movement_commands(current_pose, target_pose, current_distance, target_distance,live_corners,target_2d_corners)
                    
                    # Draw the detected marker
                    cv2.aruco.drawDetectedMarkers(frame, live_corners)
                    
                    # Draw the axis for the marker
                    try:
                        cv2.aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.05)
                        
                    except AttributeError:
                        # Fallback option if drawAxis is not available
                        cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.05)
                    
                    # Display the yaw, pitch, roll, and command on the frame
                    yaw, pitch, roll = current_pose
                    cv2.putText(frame, f'Yaw: {yaw:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    #cv2.putText(frame, f'Pitch: {pitch:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    #cv2.putText(frame, f'Roll: {roll:.2f}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    cv2.putText(frame, f'Distance: {current_distance:.2f}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    
                    if command:
                        cv2.putText(frame, f'Command: {command[0]}', (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        print("Movement Command:", command[0])
        else:
            cv2.putText(frame,'No Id detected go backward', (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
        # Display the frame
        cv2.imshow('Live Video', frame)
        
        # Check for user input to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == '__main__':
    # Load the ArUco dictionary
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
    # Create detector parameters
    parameters = cv2.aruco.DetectorParameters()

    camera_matrix = np.array([[921.170702, 0.000000, 459.904354],
                              [0.000000, 919.018377, 351.238301],
                              [0.000000, 0.000000, 1.000000]])
    dist_coeffs = np.array([-0.033458, 0.105152, 0.001256, -0.006647, 0.000000])

    # Define the size of the ArUco marker in meters
    marker_length = 0.14  # aruco marker size

    # Load baseline CSV data
    baseline_csv_file = 'output.csv'  # Replace with your baseline CSV filename
    baseline_data = load_baseline_csv(baseline_csv_file)

    # Prompt user to select target frame ID
    target_frame_id =int(input("Enter the target frame ID from the baseline CSV: "))

    # Process live video and generate movement commands
    process_live_video(baseline_data, target_frame_id)


Enter the target frame ID from the baseline CSV: 35
Webcam successfully connected.
Movement Command: up
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movem

Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Co

Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: right
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: right
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: backward
Movement Command: right
Movement Command: backward
Movement Command: right
Movement Command: right
Movement Command: backward
Movement Command: right
Movement Command: backward
Movement Command: up
Movement Command: backward
Movement Command: backwar

Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Co

Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: up
Movement Command: up
Movement Comman

Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: turn-right
Movement Command: In Postion
Movement Command: turn-right
Movement Command: turn-right
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: turn-right
Movement Command: turn-right
Movement Command: turn-right
Movement Command: turn-right
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: turn-right
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: turn-right
Movement Command: In Postion
Movement Command: In Postion
Movement Command: In Postion
Movement Command: turn-right
Movement Command: turn-right
Movement Command: In Postion
Movement Command: turn-right
Movement Command: turn-right
Movement Comma

Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Command: left
Movement Co